In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_excel('Data- acquisition scorecard.xlsx')

In [3]:
df.rename(columns={'ENQ_1':'days_last_enq','ENQ_2':'days_hl_last_enq','ENQ_3':'#_enq_3mnths','ENQ_4':'#_hle_enq_3mnths','ENQ_5':'#_enq_12mnths','ENQ_6':'#_hle_enq_12mnths','ENQ_7':'Tot_enq','ENQ_8':'Tot_hle_enq','ACCOUNT_9':'#_defaults_3mnths','ACCOUNT_10':'#_defaults_12mnths','ACCOUNT_11':'days_last_acc_open','ACCOUNT_12':'Tot_accounts','ACCOUNT_13':'Tot_hle_accounts','ACCOUNT_14':'Tot_unsecured_loans','ACCOUNT_15':'Tot_live_loans','ACCOUNT_16':'Tot_live_homeloans','ACCOUNT_17':'Tot_live_unsecuredloans','ACCOUNT_18':'Tot_amt_outstanding','ACCOUNT_19':'Tot_secured_outstanding_amt','ACCOUNT_20':'Tot_unsecured_outstanding_amt','ACCOUNT_21':'Avg_days_in_debt'},inplace=True)

In [4]:
df_copy = df.copy()

In [5]:
df.head(2)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt
0,1794,40.000,SALARIED,70891.000,62491.000,MALE,OTHERS,MARRIED,229.000,108,INDIVIDUAL,37806.000,0.190,0,324,0,0,0,1,0,14,0,0,0,1486,9,0,8,2,0,2,275606,0,275606,5311
1,3183,35.000,SALARIED,53617.000,53617.000,MALE,OTHERS,MARRIED,408.000,60,INDIVIDUAL,15499.000,0.420,0,616,1978,0,0,0,0,11,1,5,5,1912,8,0,6,1,0,0,4844,4824,20,5289


## Data Preparation

### Duplicated

In [6]:
len(df)

3980

In [7]:
df.ID.nunique()

3894

In [8]:
df.drop(df[df.duplicated()].index,inplace=True)

### Missing

In [9]:
df.isnull().sum().sort_values(ascending=False).head(7)

Bank_balance        626
Type_of_industry      5
SEX                   4
Marital_Status        4
Months_in_city        4
Age                   1
ID                    0
dtype: int64

In [10]:
sal_bb = df.loc[df['Customer_type'] == 'SALARIED','Bank_balance'].mean()
se_bb = df.loc[df['Customer_type'] == 'SELF EMPLOYED','Bank_balance'].mean()
ne_bb = df.loc[df['Customer_type'] == 'NON EARNING','Bank_balance'].mean()

In [11]:
df['Bank_balance'] = np.where((df['Customer_type'] == 'SALARIED') & (df['Bank_balance'].isna()),sal_bb,np.where(((df['Customer_type'] == 'SELF EMPLOYED')&(df['Bank_balance'].isna())),se_bb,np.where(((df['Customer_type'] == 'NON EARNING') & (df['Bank_balance'].isna())),ne_bb,df['Bank_balance'])))

In [12]:
df.isnull().sum().sort_values(ascending=False).head(7)

Type_of_industry       5
SEX                    4
Marital_Status         4
Months_in_city         4
Age                    1
ID                     0
Tot_unsecured_loans    0
dtype: int64

In [13]:
df.dropna(inplace=True)

## Variable creation

In [14]:
df.head(1)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt
0,1794,40.000,SALARIED,70891.000,62491.000,MALE,OTHERS,MARRIED,229.000,108,INDIVIDUAL,37806.000,0.190,0,324,0,0,0,1,0,14,0,0,0,1486,9,0,8,2,0,2,275606,0,275606,5311


In [15]:
dt = df.dtypes.to_frame()

In [16]:
non_obj = dt.loc[dt[0] != 'object',:].index.tolist()

In [17]:
non_obj.remove('ID')
non_obj.remove('Target')

In [19]:
for i in non_obj:
    for ii in non_obj:
        if ii != i:
            df.loc[:,'{}/{}'.format(i,ii)] = df[i]/df[ii]
        else:
            continue

In [20]:
dt = df.dtypes.to_frame()

In [21]:
df.head(2)

,ID,Age,Customer_type,Gross_income,Net_income,SEX,Type_of_industry,Marital_Status,Months_in_city,Months_in_current_job,Org_Type,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt,Age/Gross_income,Age/Net_income,Age/Months_in_city,Age/Months_in_current_job,Age/Bank_balance,Age/Debt_ratio,Age/days_last_enq,Age/days_hl_last_enq,Age/#_enq_3mnths,Age/#_hle_enq_3mnths,Age/#_enq_12mnths,Age/#_hle_enq_12mnths,Age/Tot_enq,Age/Tot_hle_enq,Age/#_defaults_3mnths,Age/#_defaults_12mnths,Age/days_last_acc_open,Age/Tot_accounts,Age/Tot_hle_accounts,Age/Tot_unsecured_loans,Age/Tot_live_loans,Age/Tot_live_homeloans,Age/Tot_live_unsecuredloans,Age/Tot_amt_outstanding,Age/Tot_secured_outstanding_amt,Age/Tot_unsecured_outstanding_amt,Age/Avg_days_in_debt,Gross_income/Age,Gross_income/Net_income,Gross_income/Months_in_city,Gross_income/Months_in_current_job,Gross_income/Bank_balance,Gross_income/Debt_ratio,Gross_income/days_last_enq,Gross_income/days_hl_last_enq,Gross_income/#_enq_3mnths,Gross_income/#_hle_enq_3mnths,Gross_income/#_enq_12mnths,Gross_income/#_hle_enq_12mnths,Gross_income/Tot_enq,Gross_income/Tot_hle_enq,Gross_income/#_defaults_3mnths,Gross_income/#_defaults_12mnths,Gross_income/days_last_acc_open,Gross_income/Tot_accounts,Gross_income/Tot_hle_accounts,Gross_income/Tot_unsecured_loans,Gross_income/Tot_live_loans,Gross_income/Tot_live_homeloans,Gross_income/Tot_live_unsecuredloans,Gross_income/Tot_amt_outstanding,Gross_income/Tot_secured_outstanding_amt,Gross_income/Tot_unsecured_outstanding_amt,Gross_income/Avg_days_in_debt,Net_income/Age,Net_income/Gross_income,Net_income/Months_in_city,Net_income/Months_in_current_job,Net_income/Bank_balance,Net_income/Debt_ratio,Net_income/days_last_enq,Net_income/days_hl_last_enq,Net_income/#_enq_3mnths,Net_income/#_hle_enq_3mnths,Net_income/#_enq_12mnths,Net_income/#_hle_enq_12mnths,Net_income/Tot_enq,Net_income/Tot_hle_enq,Net_income/#_defaults_3mnths,Net_income/#_defaults_12mnths,Net_income/days_last_acc_open,Net_income/Tot_accounts,Net_income/Tot_hle_accounts,Net_income/Tot_unsecured_loans,Net_income/Tot_live_loans,Net_income/Tot_live_homeloans,Net_income/Tot_live_unsecuredloans,Net_income/Tot_amt_outstanding,Net_income/Tot_secured_outstanding_amt,Net_income/Tot_unsecured_outstanding_amt,Net_income/Avg_days_in_debt,Months_in_city/Age,Months_in_city/Gross_income,Months_in_city/Net_income,Months_in_city/Months_in_current_job,Months_in_city/Bank_balance,Months_in_city/Debt_ratio,Months_in_city/days_last_enq,Months_in_city/days_hl_last_enq,Months_in_city/#_enq_3mnths,Months_in_city/#_hle_enq_3mnths,Months_in_city/#_enq_12mnths,Months_in_city/#_hle_enq_12mnths,Months_in_city/Tot_enq,Months_in_city/Tot_hle_enq,Months_in_city/#_defaults_3mnths,Months_in_city/#_defaults_12mnths,Months_in_city/days_last_acc_open,Months_in_city/Tot_accounts,Months_in_city/Tot_hle_accounts,Months_in_city/Tot_unsecured_loans,Months_in_city/Tot_live_loans,Months_in_city/Tot_live_homeloans,Months_in_city/Tot_live_unsecuredloans,Months_in_city/Tot_amt_outstanding,Months_in_city/Tot_secured_outstanding_amt,Months_in_city/Tot_unsecured_outstanding_amt,Months_in_city/Avg_days_in_debt,Months_in_current_job/Age,Months_in_current_job/Gross_income,Months_in_current_job/Net_income,Months_in_current_job/Months_in_city,Months_in_current_job/Bank_balance,Months_in_current_job/Debt_ratio,Months_in_current_job/days_last_enq,Months_in_current_job/days_hl_last_enq,Months_in_current_job/#_enq_3mnths,Months_in_current_job/#_hle_enq_3mnths,Months_in_current_job/#_enq_12mnths,Months_in_current_job/#_hle_enq_12mnths,Months_in_current_job/Tot_enq,Months_in_current_job/Tot_hle_enq,Months_in_current_j

In [22]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [23]:
non_obj = dt.loc[dt[0] != 'object',:].index.tolist()

In [24]:
non_obj.remove('ID')
non_obj.remove('Target')

In [25]:
non_obj_iv = pd.DataFrame()
for i in non_obj:
    temp = df.loc[:,[i,'Target']]
    temp['qcut'] = pd.qcut(temp[i],10,duplicates='drop')
    res = pd.crosstab(temp['qcut'],temp['Target'],normalize='columns').assign(woe=lambda x:np.log(x[0]/x[1])).assign(iv=lambda x:x['woe']*(x[0]-x[1]))
    res['name'] = i
    non_obj_iv = pd.concat([non_obj_iv,res])

In [26]:
obj = dt.loc[dt[0] == 'object'][0].index.tolist()

In [27]:
obj_iv = pd.DataFrame()
for i in obj:
    res = pd.crosstab(df[i],df['Target'],normalize='columns').assign(woe=lambda x:np.log(x[0]/x[1])).assign(iv=lambda x: x['woe']*(x[0]-x[1]))
    res['name'] = i
    obj_iv = pd.concat([obj_iv,res])

In [28]:
iv = pd.concat([non_obj_iv,obj_iv])

In [29]:
iv.rename(columns={0:'%good',1:'%bad'},inplace=True)

In [30]:
iv.drop(iv[iv['iv'].isin([np.inf,-np.inf])].index,inplace=True)

In [31]:
iv.sort_values(by='iv',ascending=False).head(20)

Target,%good,%bad,woe,iv,name
"(7.0, 57.0]",0.060,0.271,-1.503,0.316,#_enq_12mnths/#_hle_enq_3mnths
"(4.0, 23.0]",0.058,0.243,-1.426,0.263,#_enq_3mnths/#_defaults_3mnths
"(6.47, 52.0]",0.083,0.292,-1.256,0.262,#_enq_12mnths/Tot_hle_accounts
"(7.0, 52.0]",0.081,0.269,-1.197,0.225,#_enq_12mnths/Tot_live_homeloans
"(3.5, 24.0]",0.081,0.269,-1.197,0.225,#_enq_3mnths/Tot_live_homeloans
"(-0.001, 0.5]",0.241,0.067,1.284,0.224,#_enq_12mnths/Tot_hle_accounts
"(679187.1, 22513754.0]",0.077,0.257,-1.205,0.217,Tot_unsecured_outstanding_amt/#_defaults_3mnths
"(3.0, 23.0]",0.056,0.215,-1.347,0.215,#_enq_3mnths/#_hle_enq_3mnths
"(-0.001, 0.5]",0.216,0.058,1.319,0.208,#_enq_12mnths/Tot_live_homeloans
"(3.0, 24.0]",0.072,0.242,-1.216,0.207,#_enq_3mnths/Tot_hle_accounts


In [32]:
list(set(iv.loc[iv['iv'] == 0,'name'].values.tolist()))

['days_last_acc_open/#_defaults_3mnths',
 'Tot_amt_outstanding/#_defaults_3mnths',
 'Avg_days_in_debt/#_defaults_3mnths',
 'Net_income/#_defaults_3mnths',
 'Gross_income/#_defaults_3mnths',
 'Tot_hle_enq/#_defaults_3mnths',
 '#_defaults_3mnths/#_defaults_12mnths',
 'Bank_balance/#_defaults_3mnths',
 'Tot_unsecured_outstanding_amt/#_defaults_3mnths',
 'Months_in_current_job/#_defaults_3mnths']

In [33]:
def get_nunique(lst,num):
    res = set()
    for i in lst:
        if i in res:
            continue
        res.add(i)
        yield i
        if len(res) == num:
            return

In [34]:
temp = get_nunique(list(set(iv.sort_values(by='iv')['name'])),774)

In [35]:
temp = list(temp)

In [36]:
df.drop(columns=temp,inplace=True)

### Decision tree

In [37]:
df = pd.get_dummies(df)

In [38]:
copy_ = df_copy.copy()

In [39]:
copy_ = pd.get_dummies(copy_)

In [40]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [41]:
df.isnull().sum().sort_values(ascending=False).head()

Tot_accounts/#_defaults_12mnths                        2957
Tot_live_homeloans/Tot_secured_outstanding_amt         1712
Tot_live_unsecuredloans/Tot_secured_outstanding_amt    1712
Gross_income/Tot_live_unsecuredloans                   1578
Months_in_current_job/days_hl_last_enq                 1149
dtype: int64

In [42]:
cols = copy_.columns.tolist()
cols.remove('ID')
cols.remove('Target')

In [43]:
model_performance = pd.DataFrame()
for i in cols:
    for ii in cols:
        if i != ii:
            x = df.loc[:,[i,ii]]
            y = df.loc[:,'Target']
            x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)
            model = DecisionTreeClassifier(criterion='entropy')
            model.fit(x_train,y_train)
            pred = model.predict(x_test)
            c = {}
            c['var1'] = [i]
            c['var2'] = [ii]
            c['accuracy'] = [accuracy_score(y_test,pred)]
            model_performance = pd.concat([model_performance,pd.DataFrame.from_dict(c)])

KeyError: "None of [Index(['Age', 'Gross_income'], dtype='object')] are in the [columns]"

In [88]:
model_performance.sort_values(by='accuracy',ascending=False).head()

,var1,var2,accuracy
0,Tot_hle_enq,Customer_type_SALARIED,0.950
0,Type_of_industry_PHARMACEUTICALS BULK DRUGS,#_enq_12mnths,0.949
0,Type_of_industry_OTHER MANUFACTURING,Type_of_industry_PAPER & PULP,0.949
0,Customer_type_SELF EMPLOYED,Type_of_industry_IT AND COMMUNICATION,0.949
0,Type_of_industry_ENGINEERING,Marital_Status_WIDOWED,0.948


In [110]:
og_cols = df_copy.columns.tolist()

In [99]:
new_cols = og_cols+temp

## Random forest

In [118]:
len(df.columns)

887

In [117]:
df.isnull().sum().sort_values(ascending=False)

days_last_acc_open/#_defaults_3mnths                           3335
Tot_hle_accounts/#_defaults_3mnths                             3335
#_defaults_12mnths/#_defaults_3mnths                           3335
Tot_secured_outstanding_amt/#_defaults_3mnths                  3335
Months_in_current_job/#_defaults_3mnths                        3335
#_enq_3mnths/#_defaults_3mnths                                 3335
Tot_accounts/#_defaults_3mnths                                 3335
Months_in_city/#_defaults_3mnths                               3335
Tot_unsecured_loans/#_defaults_3mnths                          3335
Tot_hle_enq/#_defaults_3mnths                                  3335
Net_income/#_defaults_3mnths                                   3335
Tot_amt_outstanding/#_defaults_3mnths                          3335
Tot_live_loans/#_defaults_3mnths                               3335
Tot_live_homeloans/#_defaults_3mnths                           3335
Age/#_defaults_3mnths                           

In [113]:
rf_model = RandomForestClassifier(criterion='entropy')

In [115]:
df.head(2)

,ID,Age,Gross_income,Net_income,Months_in_city,Months_in_current_job,Bank_balance,Debt_ratio,Target,days_last_enq,days_hl_last_enq,#_enq_3mnths,#_hle_enq_3mnths,#_enq_12mnths,#_hle_enq_12mnths,Tot_enq,Tot_hle_enq,#_defaults_3mnths,#_defaults_12mnths,days_last_acc_open,Tot_accounts,Tot_hle_accounts,Tot_unsecured_loans,Tot_live_loans,Tot_live_homeloans,Tot_live_unsecuredloans,Tot_amt_outstanding,Tot_secured_outstanding_amt,Tot_unsecured_outstanding_amt,Avg_days_in_debt,Age/Gross_income,Age/Net_income,Age/Months_in_city,Age/Months_in_current_job,Age/Bank_balance,Age/Debt_ratio,Age/days_last_enq,Age/days_hl_last_enq,Age/#_enq_3mnths,Age/#_hle_enq_3mnths,Age/#_enq_12mnths,Age/#_hle_enq_12mnths,Age/Tot_enq,Age/Tot_hle_enq,Age/#_defaults_3mnths,Age/#_defaults_12mnths,Age/days_last_acc_open,Age/Tot_accounts,Age/Tot_hle_accounts,Age/Tot_unsecured_loans,Age/Tot_live_loans,Age/Tot_live_homeloans,Age/Tot_live_unsecuredloans,Age/Tot_amt_outstanding,Age/Tot_secured_outstanding_amt,Age/Tot_unsecured_outstanding_amt,Age/Avg_days_in_debt,Gross_income/Age,Gross_income/Net_income,Gross_income/Months_in_city,Gross_income/Months_in_current_job,Gross_income/Bank_balance,Gross_income/Debt_ratio,Gross_income/days_last_enq,Gross_income/days_hl_last_enq,Gross_income/#_enq_3mnths,Gross_income/#_hle_enq_3mnths,Gross_income/#_enq_12mnths,Gross_income/#_hle_enq_12mnths,Gross_income/Tot_enq,Gross_income/Tot_hle_enq,Gross_income/#_defaults_3mnths,Gross_income/#_defaults_12mnths,Gross_income/days_last_acc_open,Gross_income/Tot_accounts,Gross_income/Tot_hle_accounts,Gross_income/Tot_unsecured_loans,Gross_income/Tot_live_loans,Gross_income/Tot_live_homeloans,Gross_income/Tot_live_unsecuredloans,Gross_income/Tot_amt_outstanding,Gross_income/Tot_secured_outstanding_amt,Gross_income/Tot_unsecured_outstanding_amt,Gross_income/Avg_days_in_debt,Net_income/Age,Net_income/Gross_income,Net_income/Months_in_city,Net_income/Months_in_current_job,Net_income/Bank_balance,Net_income/Debt_ratio,Net_income/days_last_enq,Net_income/days_hl_last_enq,Net_income/#_enq_3mnths,Net_income/#_hle_enq_3mnths,Net_income/#_enq_12mnths,Net_income/#_hle_enq_12mnths,Net_income/Tot_enq,Net_income/Tot_hle_enq,Net_income/#_defaults_3mnths,Net_income/#_defaults_12mnths,Net_income/days_last_acc_open,Net_income/Tot_accounts,Net_income/Tot_hle_accounts,Net_income/Tot_unsecured_loans,Net_income/Tot_live_loans,Net_income/Tot_live_homeloans,Net_income/Tot_live_unsecuredloans,Net_income/Tot_amt_outstanding,Net_income/Tot_secured_outstanding_amt,Net_income/Tot_unsecured_outstanding_amt,Net_income/Avg_days_in_debt,Months_in_city/Age,Months_in_city/Gross_income,Months_in_city/Net_income,Months_in_city/Months_in_current_job,Months_in_city/Bank_balance,Months_in_city/Debt_ratio,Months_in_city/days_last_enq,Months_in_city/days_hl_last_enq,Months_in_city/#_enq_3mnths,Months_in_city/#_hle_enq_3mnths,Months_in_city/#_enq_12mnths,Months_in_city/#_hle_enq_12mnths,Months_in_city/Tot_enq,Months_in_city/Tot_hle_enq,Months_in_city/#_defaults_3mnths,Months_in_city/#_defaults_12mnths,Months_in_city/days_last_acc_open,Months_in_city/Tot_accounts,Months_in_city/Tot_hle_accounts,Months_in_city/Tot_unsecured_loans,Months_in_city/Tot_live_loans,Months_in_city/Tot_live_homeloans,Months_in_city/Tot_live_unsecuredloans,Months_in_city/Tot_amt_outstanding,Months_in_city/Tot_secured_outstanding_amt,Months_in_city/Tot_unsecured_outstanding_amt,Months_in_city/Avg_days_in_debt,Months_in_current_job/Age,Months_in_current_job/Gross_income,Months_in_current_job/Net_income,Months_in_current_job/Months_in_city,Months_in_current_job/Bank_balance,Months_in_current_job/Debt_ratio,Months_in_current_job/days_last_enq,Months_in_current_job/days_hl_last_enq,Months_in_current_job/#_enq_3mnths,Months_in_current_job/#_hle_enq_3mnths,Months_in_current_job/#_enq_12mnths,Months_in_current_job/#_hle_enq_12mnths,Months_in_current_job/Tot_enq,Months_in_current_job/Tot_hle_enq,Months_in_current_job/#_defaults_3mnths,Months_in_current_job/#_defaults_12mnt

In [114]:
x = df.drop(columns='Target')

In [ ]:
rf_model.fit

### Variable clustering

```
pip install VarClusHi
```

```python
from varclushi import VarClusHi
df.shape
x = df.drop(columns='Target')
x = x[x.describe().columns.tolist()]
vc_model = VarClusHi(x,maxegival2 = 1,maxclus=2)